In [20]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#Defining region of interest function
def roi(img, vertices):
    mask = np.zeros_like(img)
    mask_color = 255
    cv2.fillPoly(mask, vertices, mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

#Defining draw the line function
def draw_line(img, lines):
    c_img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1),(x2,y2), (255, 0, 0), thickness=3)
    
    #To merge original and blank image        
    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img



def process(image):
    #Defining the area of interest
    triangle = [
                (0, 540),
                (960/2, 300),
                (960, 540)
    ]
    t_vertices = np.array([triangle], dtype=np.int32)
    
    #Creating a gray scale image
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    #Creating a Canny image 
    lower_threshold = 200 
    upper_threshold = 250
    canny_image = cv2.Canny(gray, lower_threshold, upper_threshold)
   
    #Calling the function roi
    cropped_image = roi(canny_image, t_vertices )

    #Applying Hough transform
    lines = cv2.HoughLinesP(cropped_image,
                          rho = 1,
                          theta=np.pi/180,
                          threshold=10,
                          lines=np.array([]),
                          minLineLength = 10,
                          maxLineGap=20)
    image_with_lines = draw_line(image, lines)
    return image_with_lines

white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


t:   2%|▏         | 4/221 [00:00<00:05, 38.39it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 8.62 s


In [21]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))
